## Supervised learning 

In [76]:
# import the appropriate libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut


removing irrelevant features ?-> Chi-squared // not done in exercises

standardization

on va faire du cross validation parce qu'on a un petit dataset
quand même besoin de garder un set de côté pour après ou paaasssss ???

Methods 
- kNN
    - no training needed
    - accuracy improves with more data -> AHHH
    - Hyperparameters: k and distance fct
    - Choose k with LOO crossvalidation
    
- decision trees, random forest
    - Random forest good results and easy to implement!!

- Linear and logistic regression
    - use ridge regression to already have regularization
    - get_dummies to convert sex to numerical (série 6)

- convolutionnal neural network
    - pas vu en cours mais donne bons résultats normalement 

asses results : confusion matrix -> precision and recall -> precision recall curve or ROC

In [56]:
# load data
data = pd.read_csv('merge_features_filtered.csv')
data.head()

,Trope,CharacterName,ActorName,ActorDOB,MovieName,Genres,Decade,ReleaseDate,BoxOffice,ActorID,...,actorID=341,actorID=342,actorID=343,Eye Distance,Eye Position,Nose Length,Nose Width,Eyebrow Length,Face Shape,Cheek Bones
0,evil_character,Aaron Hallam,Benicio del Toro,1967.0,The Hunted,"['Thriller', 'Crime Fiction', 'Action', 'Drama']",2000.0,2003.0,45496534.0,137.0,...,0.892761,0.820569,0.772236,0.267677,0.948980,0.373469,0.200337,0.340909,1.212245,0.183502
1,mean_officer,Abel Turner,Samuel L. Jackson,1948.0,Lakeview Terrace,"['Thriller', 'Crime Thriller', 'Drama', 'Psych...",2000.0,2008.0,44653637.0,273.0,...,1.058803,0.971612,1.004091,0.286885,0.949698,0.344064,0.260656,0.336885,1.227364,0.198361
2,charismatic_charmer,Abu,Sabu Dastagir,1924.0,The Thief of Bagdad,"['Adventure', 'Costume drama', 'Action/Adventu...",1940.0,1940.0,NaN,72.0,...,0.867241,0.812289,0.863133,0.261780,0.961039,0.396104,0.183246,0.348168,1.240260,0.170157
3,charismatic_charmer,Achilles,Brad Pitt,1963.0,Troy,"['Sword and Sandal', 'Adventure', 'Costume dra...",2000.0,2004.0,497409852.0,193.0,...,0.842054,0.782616,0.786956,0.264151,0.939914,0.347639,0.188679,0.328302,1.137339,0.141509
4,laidback_freebird,Agatha,Samantha Morton,1977.0,Minority Report,"['Crime Fiction', 'Thriller', 'Science Fiction...",2000.0,2002.0,358372926.0,178.0,...,0.872403,0.907091,0.919244,0.244406,0.931193,0.410550,0.175559,0.327022,1.332569,0.166093


In [57]:
data.columns

Index(['Trope', 'CharacterName', 'ActorName', 'ActorDOB', 'MovieName',
       'Genres', 'Decade', 'ReleaseDate', 'BoxOffice', 'ActorID',
       ...
       'actorID=341', 'actorID=342', 'actorID=343', 'Eye Distance',
       'Eye Position', 'Nose Length', 'Nose Width', 'Eyebrow Length',
       'Face Shape', 'Cheek Bones'],
      dtype='object', length=493)

removed actorDOB (have age) and ethnicity (shitty data)

In [58]:
filtered_data = data[['Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice', 'ActorID', 'ActorGender', 'ActorHeight', 'ActorAge', 'Eye Position', 'Nose Length', 'Nose Width', 'Eyebrow Length', 'Face Shape', 'Cheek Bones']]
filtered_data.head()

,Trope,CharacterName,ActorName,Genres,ReleaseDate,BoxOffice,ActorID,ActorGender,ActorHeight,ActorAge,Eye Position,Nose Length,Nose Width,Eyebrow Length,Face Shape,Cheek Bones
0,evil_character,Aaron Hallam,Benicio del Toro,"['Thriller', 'Crime Fiction', 'Action', 'Drama']",2003.0,45496534.0,137.0,M,1.85,36.0,0.948980,0.373469,0.200337,0.340909,1.212245,0.183502
1,mean_officer,Abel Turner,Samuel L. Jackson,"['Thriller', 'Crime Thriller', 'Drama', 'Psych...",2008.0,44653637.0,273.0,M,1.89,60.0,0.949698,0.344064,0.260656,0.336885,1.227364,0.198361
2,charismatic_charmer,Abu,Sabu Dastagir,"['Adventure', 'Costume drama', 'Action/Adventu...",1940.0,NaN,72.0,M,NaN,16.0,0.961039,0.396104,0.183246,0.348168,1.240260,0.170157
3,charismatic_charmer,Achilles,Brad Pitt,"['Sword and Sandal', 'Adventure', 'Costume dra...",2004.0,497409852.0,193.0,M,1.80,41.0,0.939914,0.347639,0.188679,0.328302,1.137339,0.141509
4,laidback_freebird,Agatha,Samantha Morton,"['Crime Fiction', 'Thriller', 'Science Fiction...",2002.0,358372926.0,178.0,F,1.60,25.0,0.931193,0.410550,0.175559,0.327022,1.332569,0.166093


In [59]:
# X_data = filtered_data[['ActorGender', 'ActorHeight', 'ActorAge', 'Eye Position', 'Nose Length', 'Nose Width', 'Eyebrow Length', 'Face Shape', 'Cheek Bones']]
X_data = filtered_data[['ActorGender', 'ActorAge', 'Eye Position', 'Nose Length', 'Nose Width', 'Eyebrow Length', 'Face Shape', 'Cheek Bones']]
X_num_data = pd.get_dummies(X_data)
y_data = filtered_data[['Trope']]
len = y_data.size
y_data = y_data.values.reshape(len,)

In [64]:
#simple KNN implementation
K = 3
KNN_Classifier = KNeighborsClassifier(n_neighbors=K)
KNN_Classifier.fit(X_num_data, y_data)

y_pred = cross_val_predict(KNN_Classifier, X_num_data, y_data, cv=5)
report = classification_report(y_data, y_pred)
print(report)

                     precision    recall  f1-score   support

charismatic_charmer       0.23      0.46      0.31        54
      crazy_fighter       0.08      0.17      0.11        36
  crazy_jealous_guy       0.05      0.08      0.06        25
    dumb_and_clumsy       0.19      0.31      0.24        36
   emotional_damage       0.10      0.06      0.07        17
     evil_character       0.07      0.08      0.07        26
               jock       0.00      0.00      0.00        27
  laidback_freebird       0.55      0.34      0.42        35
              loser       0.00      0.00      0.00        25
       mean_officer       0.10      0.06      0.08        48
    old_wise_quirky       0.27      0.17      0.21        24
   respected_leader       0.08      0.04      0.05        25
shallow_and_popular       0.43      0.39      0.41        31
           sidekick       0.00      0.00      0.00        21
     skilled_badass       0.00      0.00      0.00        50
        tech_genius    

In [77]:
#KNN with tuning of K

k_range = list(range(1, 31))

# Create a parameter grid: map the parameter names to the values that should be searched
param_grid = dict(n_neighbors=k_range)

# perform the search with CV
loo = LeaveOneOut()
grid = GridSearchCV(KNN_Classifier, param_grid, cv=loo, scoring='accuracy')
grid.fit(X_num_data, y_data)
results = grid.cv_results_

# Print the best score and the best K
print("Best score: %0.2f" % grid.best_score_)
best_K = grid.best_estimator_.get_params()['n_neighbors']
print("Best K:", best_K)

Best score: 0.20
Best K: 1


In [78]:
# Use the best parameters to fit the KNN model
KNN_Classifier = KNeighborsClassifier(n_neighbors=best_K)
KNN_Classifier.fit(X_num_data, y_data)

# Make predictions
y_pred = cross_val_predict(KNN_Classifier, X_num_data, y_data, cv=5)

report = classification_report(y_data, y_pred)
print(report)

                     precision    recall  f1-score   support

charismatic_charmer       0.26      0.22      0.24        54
      crazy_fighter       0.09      0.11      0.10        36
  crazy_jealous_guy       0.04      0.04      0.04        25
    dumb_and_clumsy       0.17      0.17      0.17        36
   emotional_damage       0.20      0.24      0.22        17
     evil_character       0.06      0.08      0.07        26
               jock       0.08      0.07      0.08        27
  laidback_freebird       0.51      0.51      0.51        35
              loser       0.05      0.04      0.04        25
       mean_officer       0.11      0.10      0.11        48
    old_wise_quirky       0.40      0.25      0.31        24
   respected_leader       0.06      0.08      0.07        25
shallow_and_popular       0.29      0.29      0.29        31
           sidekick       0.15      0.14      0.15        21
     skilled_badass       0.11      0.12      0.12        50
        tech_genius    